In [1]:
from board import Board
import numpy as np
import random

In [2]:
board = Board(2, 2)

In [29]:
def generate_hidden_single_row(board, x, y, digit, filled=5):
    """
    If want to generate column, just take the transpose
    """
    board = board.copy()
    board.write(x, y, digit)
    filled -= np.sum(board[x] > 0)
    
    unused_digits = set(np.arange(board.max_digit) + 1) - set(board[x][board[x] > 0])
    empty_ys = np.nonzero(board[x] == 0)[0]
    np.random.shuffle(empty_ys)
    
    for empty_y in empty_ys:
        if filled == 0:
            break
        fill_digit = random.sample(unused_digits, 1)[0]
        board.write(x, empty_y, fill_digit)
        unused_digits.remove(fill_digit)
        filled -= 1
    return board

def generate_hidden_single(board, x, y, digit, house='row', filled=5, ):
    assert house in ['row', 'column', 'box']
    

In [30]:
board = Board(3, 3)
generate_hidden_single_row(board, 3, 3, 3)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [9, 0, 6, 3, 0, 0, 0, 4, 7],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [11]:
random.sample({4,3,2}, 1)[0]

4